In [ ]:
!pip install -q geopandas;
!pip install -q geemap;
!pip install -q --upgrade folium;
# !pip install sentinelhub;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 456.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.0 MB/s eta 0:00:00
  

In [ ]:
import json
import ee, datetime, folium
import pandas as pd
from IPython.display import Image
from pylab import *
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from folium.plugins import HeatMap
import io
import fiona
import branca.colormap as cm
import geemap
# import geemap.foliumap
# import geemap.eefolium as geemap
from shapely.geometry import shape
from shapely.geometry import shape, Polygon, MultiPolygon

# Extract Satellite Indices per Geographic Area in England

## First authenticate GEE

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=H_zfM9b5A-TKUhDGs3H6Y5kFsTcfpDOcFEGDKB0NxdQ&tc=yJenfGXbTadNN2xDmSe8tjOzIYCbE3o5NELPnSYGf6Q&cc=wb8olrJKcRh6G_obRIS8Fmcc0Vetf4t3EamvbHzKdfc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPHI55z-KlVTbjhZXbM9DzQfS0miDQ60eSW1P0zFMT8SHMo1sYh9cs

Successfully saved authorization token.


## Then also connect to Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Global Variables

In [ ]:
project_folder = "drive/MyDrive/your_folder/"

## 0 Functions

In [ ]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Define a function for displaying shapefiles saved into a feature collection
def plot_ee_feature_shapefile(ee_feature, zoom_start=7):
  # Get the bounds of the feature collection
  aoi = ee_feature.geometry()

  # Create a folium map object.
  center = aoi.centroid(10).coordinates().reverse().getInfo()
  m = folium.Map(location=center, zoom_start=zoom_start)

  # Add layers to the folium map.
  # Add the feature collection to the map as a GeoJSON layer
  m.add_child(folium.GeoJson(data=ee_feature.getInfo()))

  # Add a layer control panel to the map.
  m.add_child(folium.LayerControl())

  # Display the map.
  display(m)

In [ ]:
def plot_ee_feature_shapefile_geemap(fc, zoom_start=6, coordinates=(-0.1278, 51.5074), color=None):

  # Create a Map object
  Map = geemap.Map(center=[0, 0], zoom=zoom_start) # set initial center and zoom level

  # Add the Feature Collection to the Map, centered on the shapefile
  Map.addLayer(fc, {}, 'Shapefile')

  # too many edges say GEE
  # center = fc.geometry().centroid(100).coordinates().reverse().getInfo()
  # print (center)

  # # Automatically adjust the zoom level to fit the shapefile
  # Map.setCenter(lon=coordinates[0], lat=coordinates[1]) # center on the centroid

  Map.centerObject(fc, zoom_start)

  if color:
    # Add a color based on the 'means' property
    colorized = fc.map(lambda feature: feature.set({'style': {'color': 'blue', 'fillColor': feature.get(color)}}))

  # Add the colorized feature collection to the map
  Map.addLayer(colorized, {}, 'Colorized')

  display(Map)

In [ ]:
def spatial_join_ee_fc(ee_fc_fine, ee_fc_coarse):

  # Define the spatial join relationship
  spatialFilter = ee.Filter.intersects(
    leftField='.geo', # geometry field in regions feature collection
    rightField='.geo', # geometry field in wards feature collection
    maxError=10) # tolerance distance in meters

  # Perform the spatial join using the defined spatial filter
  ee_fc_joined = ee.Join.simple().apply(
    primary=ee_fc_fine,
    secondary=ee_fc_coarse,
    condition=spatialFilter)

  return ee_fc_joined

In [ ]:
def get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE):

  # these are parameters for selecting the Sentinel Image
  aoi = ee_the_region.geometry()

  # START_DATE = '2017-01-01' # inclusive
  # END_DATE = '2018-04-01' # exclusive
  CLOUD_FILTER = 5 # maximum amount of clouds in percent

  # Import and filter S2 collection
  s2_col = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(aoi)
      .filterDate(START_DATE, END_DATE)
      .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

  return s2_col

In [ ]:
def calculate_ndvi(image):
    # Calculate NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Define the threshold
    threshold = 0.2

    # Create a binary mask using the threshold
    ndvi_binary = ndvi.gt(threshold).toFloat().rename('NDVI_binary')

    filled_image = ndvi_binary.unmask(0)

    return filled_image


In [ ]:
def efficent_fc_to_gdf(fc, selectors, batch_size=2000):

  print ("running efficient conversion to gdf")

  # Get the total number of features in the feature collection
  num_features = fc.size().getInfo()

  # Define a list to store the GeoDataFrames
  gdfs = []

  # Loop over the feature collection in batches of size batch_size
  for i in range(0, num_features, batch_size):
      print ("running efficient conversion to gdf. data {} of {}...".\
              format(i, int(num_features)))
      # Get the next batch of features using the limit() method
      # features = fc.limit(batch_size, str(i))
      features_list = fc.toList(batch_size, i)
      features = ee.FeatureCollection(features_list)

      # Convert the batch of features to a GeoDataFrame
      gdf = geemap.ee_to_geopandas(features, selectors=selectors)

      # Add the GeoDataFrame to the list
      gdfs.append(gdf)

  # Concatenate the GeoDataFrames into a single GeoDataFrame
  final_gdf = pd.concat(gdfs)

  # Display the GeoDataFrame
  # print(final_gdf.head())

  return final_gdf

In [ ]:
def extract_NVDI_lsoa_scores_region(region_name, ee_regions, ee_lsoas_UK, sat, START_DATE, END_DATE):

  year = START_DATE.split("-")[0]
  print("****** Processing {} in {} year *******".format(region_name, year))

  # prepare region geographies
  ee_the_region = ee_regions.filter(ee.Filter.eq('RGN22NM',region_name))
  ee_region_lsoas = spatial_join_ee_fc(ee_lsoas_UK, ee_the_region)
  print ("prepared Region geographies ...")

  # pull Sentinel-2 imagery
  s2 = get_satellite_imagery(ee_the_region, sat, START_DATE, END_DATE)
  print ("pulled Sentinel-2 imagery ...")

  # calculate NVDI
  s2 = s2.median()
  s2_col = ee.ImageCollection([s2])
  S2NDVI_col = s2_col.map(calculate_ndvi)

  # Get the first image from the resulting image collection
  S2NDVI_image = ee.Image(S2NDVI_col.first())
  print ("calculated Region NDVI ...")

  # take the Sentinel-2 NDVI we calculated
  ndvi = S2NDVI_image

  # get the shapefile we need
  shapefile = ee_region_lsoas
  # Get the mean value of each band within the shapefile
  means = ndvi.reduceRegions(collection=shapefile, reducer=ee.Reducer.mean(), scale=50)

  # Print the calculated LSOAs NDVI
  print("Calculated LSOAs NDVI...")

  selectors = ['LSOA21NM', 'LSOA21CD', 'mean']
  # Convert the feature collection to a GeoPandas dataframe
  gdf_means = efficent_fc_to_gdf(means, selectors=selectors)
  # gdf_means = geemap.ee_to_geopandas(means, selectors=selectors)
  # # Define the CRS of the GeoJSON data
  # gdf_means.crs = 'EPSG:4326'
  print ("converted feature collection to geopandas df ...")

  # save csv with results
  csv_results_address = project_folder + \
         "results/{}/lsoa/{}/{}.csv".format(sat, year, region_name)
  # create a Pandas dataframe with selected columns
  df_means = pd.DataFrame(gdf_means[selectors])
  df_means.to_csv(csv_results_address, index=False)
  print ("saved csv ...")

  return gdf_means

## 1 Download / Read In Data

## Spatial Data

### Region Data

In [ ]:
# England regions data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/ons::regions-december-2022-en-bfc/explore?location=52.916331%2C-2.000000%2C7.74

drive_address = project_folder + "data/England-regions/Regions_(December_2022)_EN_BFC.shp"
asset_address = "projects/sanja7s-ee/assets/England_regions"


# 1 we can read them into geopandas from Drive
regions = gpd.read_file(drive_address)
print("There are {} regions in England.".format(len(regions)))

# 2 we can read the data as a feature shapefile from GEE assets
ee_regions = ee.FeatureCollection(asset_address)

print (regions.crs)
# regions.head()

There are 9 regions in England.
EPSG:27700


### Ward Data

In [ ]:
# # UK wards data is from here https://geoportal.statistics.gov.uk/
# # https://geoportal.statistics.gov.uk/search?collection=Dataset&sort=name&tags=all(BDY_WD)

# drive_address = project_folder + "data/UK-wards/WD_DEC_2019_GB_BFE/geodb_DBO_WD_DEC_2019_GB_BFE.shp"
# asset_address = "projects/sanja7s-ee/assets/UK_wards"


# # # 1 we can read them into geopandas from Drive
# wards_UK = gpd.read_file(drive_address)
# wards = wards_UK[wards_UK['wd19cd'].str.startswith('E')]
# print("There are {} wards in England out of {} in the UK.".format(len(wards), len(wards_UK)))

# # # 2 we can read the data as a feature shapefile from GEE assets
# ee_wards_UK = ee.FeatureCollection(asset_address)

# # # 3 join wards with regions
# wards_regions = gpd.sjoin(wards, regions, predicate='within')
# ee_wards_regions = spatial_join_ee_fc(ee_wards_UK, ee_regions)

# # print (wards.crs)
# # wards.head()

In [ ]:
# plot_ee_feature_shapefile_geemap(ee_wards_regions, zoom_start=7)

### LSOAs Data

In [ ]:
# UK LSOAs data is from here https://geoportal.statistics.gov.uk/
# https://geoportal.statistics.gov.uk/datasets/766da1380a3544c5a7ca9131dfd4acb6_0/explore?location=51.420294%2C-0.114413%2C11.00

drive_address = project_folder + "data/UK-LSOAs/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).shp"
asset_address = "projects/sanja7s-ee/assets/UK_LSOAs"


# 1 we can read them into geopandas from Drive
lsoas_UK = gpd.read_file(drive_address)
lsoas = lsoas_UK[lsoas_UK['LSOA21CD'].str.startswith('E')]
print("There are {} wards in England out of {} in the UK.".format(len(lsoas), len(lsoas_UK)))

# 2 we can read the data as a feature shapefile from GEE assets
ee_lsoas_UK = ee.FeatureCollection(asset_address)
# # 3 join wards with regions
lsoas_regions = gpd.sjoin(lsoas, regions, predicate='within')
ee_lsoas_regions = spatial_join_ee_fc(ee_lsoas_UK, ee_regions)

# lsoas.head()
print (lsoas.crs)

There are 33755 wards in England out of 35672 in the UK.
EPSG:27700


In [ ]:
# plot_ee_feature_shapefile_geemap(ee_wards_regions, zoom_start=7)

# Process a Region

In [ ]:
# ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name='West Midlands',\
#                                ee_regions=ee_regions, ee_wards_UK=ee_wards_UK,\
#                             sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

# Process all regions

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#   ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name=region_name, ee_regions=ee_regions, ee_wards_UK=ee_wards_UK)

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#   ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name=region_name,\
#                               ee_regions=ee_regions, ee_wards_UK=ee_wards_UK,\
#                             sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#   ward_NDVI_scores = extract_NVDI_ward_scores_region(region_name=region_name,\
#                               ee_regions=ee_regions, ee_wards_UK=ee_wards_UK,\
#                             sat='S2', START_DATE='2019-01-01', END_DATE='2020-01-01')

# Process an LSOA

In [ ]:
# lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name='London',\
#                             ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                             sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing London *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 5128...
running efficient conversion to gdf. data 2000 of 5128...
running efficient conversion to gdf. data 4000 of 5128...
converted feature collection to geopandas df ...
saved csv ...


In [ ]:
# lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name='London',\
#                             ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                             sat='S2', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing London in 2019 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 5128...
running efficient conversion to gdf. data 2000 of 5128...
running efficient conversion to gdf. data 4000 of 5128...
converted feature collection to geopandas df ...
saved csv ...


In [ ]:
# lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name='London',\
#                             ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
#                             sat='S2', START_DATE='2020-01-01', END_DATE='2021-01-01')

****** Processing London in 2020 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 5128...
running efficient conversion to gdf. data 2000 of 5128...
running efficient conversion to gdf. data 4000 of 5128...
converted feature collection to geopandas df ...
saved csv ...


# Process all LSOAs

In [ ]:
# for region_name in regions['RGN22NM'].unique():
#     lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name, ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK)

## 2021

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name,\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE='2021-01-01', END_DATE='2022-01-01')

****** Processing North East in 2021 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2021 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2021 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gd

## 2019

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name,\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE='2019-01-01', END_DATE='2020-01-01')

****** Processing North East in 2019 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2019 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2019 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gd

## 2020

In [ ]:
for region_name in regions['RGN22NM'].unique():
    lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name,\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE='2020-01-01', END_DATE='2021-01-01')

****** Processing North East in 2020 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2020 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2020 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
Calculated LSOAs NDVI...
running efficient conversion to gdf
running efficient conversion to gd

2017 - 2018 (the earliest possible with this satellite)

In [ ]:
START_DATE = '2017-01-01' # inclusive
END_DATE = '2018-04-01' # exclusive

for region_name in regions['RGN22NM'].unique():
    lsoa_NDVI_scores = extract_NVDI_lsoa_scores_region(region_name=region_name,\
                            ee_regions=ee_regions, ee_lsoas_UK=ee_lsoas_UK,\
                            sat='S2', START_DATE=START_DATE, END_DATE=END_DATE)

****** Processing North East in 2017 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 1705...
converted feature collection to geopandas df ...
saved csv ...
****** Processing North West in 2017 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to gdf. data 0 of 4642...
running efficient conversion to gdf. data 2000 of 4642...
running efficient conversion to gdf. data 4000 of 4642...
converted feature collection to geopandas df ...
saved csv ...
****** Processing Yorkshire and The Humber in 2017 year *******
prepared Region geographies ...
pulled Sentinel-2 imagery ...
calculated Region NDVI ...
calculated LSOAs NDVI ...
running efficient conversion to gdf
running efficient conversion to